# Libs

In [1]:
import pandas as pd

from pathlib import Path

from mbp.preprocessing.loader import Loader

from scripts.datasets import sit_datasets
from scripts.feature_preparation import OpenFaceFeaturePreparator, OpenFaceFeatureTransformer, AudioVisualFeatureCombiner

In [2]:
VERSION = 'acii23'
FEATURES_FILENAME = f'audiovisual_features_v_{VERSION}.csv'

In [ ]:
for dataset in sit_datasets.values():
    output_filepath = Path(dataset.processed / 'open_face_features_combined.csv')
    openface_feature_combiner = OpenFaceFeaturePreparator(dataset, output_filepath)
    openface_feature_combiner.run()

    openface_features = pd.read_csv(output_filepath)
    output_filepath.unlink()
    openface_features_transformer = OpenFaceFeatureTransformer()
    visual_features = openface_features_transformer.fit_transform(openface_features)

    loader = Loader(dataset.processed / 'open_smile_features', ['*.csv'])
    audio_feature_files = loader.load()

    output_filepath = Path(dataset.processed / FEATURES_FILENAME)

    feature_combiner = AudioVisualFeatureCombiner(visual_features, audio_feature_files, dataset.labels, output_filepath)
    feature_combiner.run()

In [ ]:
from scripts.data_objects import SITCondition

for dataset in list(sit_datasets.values()):
    
    general_path = dataset.processed / 'acii23_labels.csv'
    features = pd.read_csv(general_path)
    features = features.rename(columns={'label': SITCondition.ASD})
    
    features[['id', 'age', 'gender', 'SITCondition.ASD']].to_csv(general_path, index=False)